# Setup the model

In [1]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace

# Use HuggingFaceEndpoint - automatic routing based on availability
llm = HuggingFaceEndpoint(
    repo_id="Qwen/Qwen2.5-14B-Instruct",
    task="text-generation",
    max_new_tokens=512,
    temperature=0.1,
)

chat_llm = ChatHuggingFace(llm=llm)

In [2]:
import langchain as lc
import wikipedia as wc
print("LangChain version:", lc.__version__)
print("Wikipedia version:", wc.__version__)
!python -V

LangChain version: 1.2.6
Wikipedia version: (1, 4, 0)
Python 3.11.0


# Agent types

## Create a database

In [3]:
from sqlalchemy import MetaData

metadata_obj = MetaData()

In [4]:
from sqlalchemy import Column, Integer, String, Table, Date, Float

stocks = Table(
    "stocks",
    metadata_obj,
    Column("obs_id", Integer, primary_key=True),
    Column("stock_ticker", String(4), nullable=False),
    Column("price", Float, nullable=False),
    Column("date", Date, nullable=False),
)

In [5]:
from sqlalchemy import create_engine

engine = create_engine("sqlite:///:memory:")
metadata_obj.create_all(engine)

In [6]:
from datetime import datetime

observations = [
    [1, 'ABC', 200, datetime(2023, 1, 1)],
    [2, 'ABC', 208, datetime(2023, 1, 2)],
    [3, 'ABC', 232, datetime(2023, 1, 3)],
    [4, 'ABC', 225, datetime(2023, 1, 4)],
    [5, 'ABC', 226, datetime(2023, 1, 5)],
    [6, 'XYZ', 810, datetime(2023, 1, 1)],
    [7, 'XYZ', 803, datetime(2023, 1, 2)],
    [8, 'XYZ', 798, datetime(2023, 1, 3)],
    [9, 'XYZ', 795, datetime(2023, 1, 4)],
    [10, 'XYZ', 791, datetime(2023, 1, 5)],
]

In [7]:
from sqlalchemy import insert

def insert_obs(obs):
    stmt = insert(stocks).values(
    obs_id=obs[0],
    stock_ticker=obs[1],
    price=obs[2],
    date=obs[3]
    )

    with engine.begin() as conn:
        conn.execute(stmt)

In [8]:
for obs in observations:
    insert_obs(obs)

In [9]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase(engine)

## Agent type #1: Zero Shot React

In [10]:
from langchain_community.agent_toolkits.sql.base import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_classic.agents.agent_types import AgentType

agent_executor = create_sql_agent(
    llm=chat_llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=chat_llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    max_iterations=6
)

2026-02-20 04:57:40.898031: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
result = agent_executor.invoke(
    "What is the multiplication of the ratio between stock " +
    "prices for 'ABC' and 'XYZ' in January 3rd and the ratio " +
    "between the same stock prices in January the 4th?"
)
print(f"Result: {result['output']}")



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: stocksI now know there is a table called 'stocks'. I need to check its schema to understand what data it contains.
Action: sql_db_schema
Action Input: stocks
CREATE TABLE stocks (
	obs_id INTEGER NOT NULL, 
	stock_ticker VARCHAR(4) NOT NULL, 
	price FLOAT NOT NULL, 
	date DATE NOT NULL, 
	PRIMARY KEY (obs_id)
)

/*
3 rows from stocks table:
obs_id	stock_ticker	price	date
1	ABC	200.0	2023-01-01
2	ABC	208.0	2023-01-02
3	ABC	232.0	2023-01-03
*/Now I understand the structure of the `stocks` table. To calculate the multiplication of the ratio between stock prices for 'ABC' and 'XYZ' on January 3rd and the same ratio on January 4th, I need to extract the relevant prices for both days and compute the ratios. Afterward, I'll multiply these ratios together.
Action: sql_db_query_checker
Action Input: SELECT (s1.price / s2.price) * (s3.price / s4.price) AS ratio_multiplication FROM stocks s1 JOIN stocks s2 ON s1

In [12]:
print(agent_executor.agent.runnable.steps[1].template)

You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct sqlite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 10 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't k

## Agent type #2: Conversational React

In [13]:
# Use the modern tool-based approach instead of deprecated LLMMathChain
from langchain_core.tools import tool
import math
import numexpr

@tool
def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
        "10000 * (1 + 0.08)**5" for compound interest
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

In [14]:
from langchain_core.tools import tool

# Add a final_answer tool as recommended in the guide
@tool
def final_answer(answer: str, tools_used: list[str]) -> str:
    """Use this tool to provide a final answer to the user.
    The answer should be in natural language as this will be provided
    to the user directly. The tools_used must include a list of tool
    names that were used within the `scratchpad`.
    """
    return {"answer": answer, "tools_used": tools_used}

# Add tools
tools = [final_answer, calculator]

In [15]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# Create a simple chat history storage
chat_map = {}

def get_chat_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in chat_map:
        # if session ID doesn't exist, create a new chat history
        chat_map[session_id] = InMemoryChatMessageHistory()
    return chat_map[session_id]

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# First, create a prompt that forces the LLM to analyze the history
history_analysis_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "Analyze the conversation history below and identify any calculations that have already been performed. "
        "Extract the results of these calculations so they can be reused instead of recalculating."
    )),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "What calculations have been done and what are their results?"),
])

# Then the main agent prompt
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", (
        "You're a helpful assistant. When answering a user's question "
        "you should first use one of the tools provided. After using a "
        "tool the tool output will be provided in the "
        "'scratchpad' below. If you have an answer in the "
        "scratchpad you should not use any more tools and "
        "instead answer directly to the user. "
        "IMPORTANT: Use the analysis of previous calculations to avoid recalculating."
    )),
    ("human", "Previous calculations analysis: {history_analysis}"),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [17]:
from langchain_core.runnables.base import RunnableSerializable

# define the agent runnable with history analysis first
agent: RunnableSerializable = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: x["chat_history"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | {
        "history_analysis": lambda x: chat_llm.invoke(history_analysis_prompt.format_messages(
            chat_history=x["chat_history"]
        )).content,
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | agent_prompt
    | chat_llm.bind_tools(tools)
)

In [18]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

# create tool name to function mapping as per guide
name2tool = {tool.name: tool.func for tool in tools}

class CustomAgentExecutor:
    def __init__(self, session_id: str, max_iterations: int = 5):
        self.max_iterations = max_iterations
        self.session_id = session_id
        self.chat_history = get_chat_history(session_id)  # retrieves or creates session history
        self.agent = agent

    def invoke(self, input: str) -> dict:
        count = 0
        agent_scratchpad = []
        final_answer = None

        # print("chat history:", self.chat_history.messages)

        while count < self.max_iterations:
            try:
                tool_call = self.agent.invoke({
                    "input": input,
                    "chat_history": self.chat_history.messages,  # ← .messages here
                    "agent_scratchpad": agent_scratchpad
                })
            except Exception as e:
                print(f"Agent invocation failed at iteration {count}: {e}")
                final_answer = "I encountered an error processing your request."
                break
            
            # print(f"Agent output at iteration {count}:", tool_call)
            
            agent_scratchpad.append(tool_call)

            if tool_call.tool_calls:
                for tool_call_obj in tool_call.tool_calls:
                    tool_name = tool_call_obj["name"]
                    tool_args = tool_call_obj["args"]
                    tool_call_id = tool_call_obj["id"]

                    if tool_name not in name2tool:
                        print(f"Unknown tool called: {tool_name}, skipping...")
                        continue

                    tool_out = name2tool[tool_name](**tool_args)
                    tool_exec = ToolMessage(
                        content=f"{tool_out}",
                        tool_call_id=tool_call_id
                    )
                    agent_scratchpad.append(tool_exec)
                    print(f"{count}: {tool_name}({tool_args}) -> {tool_out}")

                count += 1

                if any(tc["name"] == "final_answer" for tc in tool_call.tool_calls):
                    final_tool_call = next(tc for tc in tool_call.tool_calls if tc["name"] == "final_answer")
                    final_answer = final_tool_call["args"]["answer"]
                    break

            else:
                final_answer = tool_call.content if tool_call.content else "No answer generated."
                break

        if final_answer is None:
            final_answer = "Could not reach a final answer within the iteration limit."

        # ← use dedicated methods instead of list.extend()
        self.chat_history.add_user_message(input)
        self.chat_history.add_ai_message(final_answer)

        return {"output": final_answer}

In [19]:
# Initialize with a session ID
conversational_agent = CustomAgentExecutor(session_id="buffer_session_1")

# First question
result = conversational_agent.invoke("What is 10000 * (1 + 0.08)**5?")
print(f"Result: {result['output']}")

0: calculator({'expression': '10000 * (1 + 0.08)**5'}) -> 14693.280768000006
1: final_answer({'answer': 'approximately 14693.28', 'tools_used': ['calculator']}) -> {'answer': 'approximately 14693.28', 'tools_used': ['calculator']}
Result: approximately 14693.28


In [20]:
print("Prompt Template Structure:")
print("=" * 50)
print("System message:", agent_prompt.messages[0].prompt.template)
print("History analysis template:", agent_prompt.messages[1].prompt.template)  # Show the memory component
print("Human message template:", agent_prompt.messages[2].prompt.template)
print("Variables:", agent_prompt.input_variables)
print("=" * 50)

Prompt Template Structure:
System message: You're a helpful assistant. When answering a user's question you should first use one of the tools provided. After using a tool the tool output will be provided in the 'scratchpad' below. If you have an answer in the scratchpad you should not use any more tools and instead answer directly to the user. IMPORTANT: Use the analysis of previous calculations to avoid recalculating.
History analysis template: Previous calculations analysis: {history_analysis}
Human message template: {input}
Variables: ['agent_scratchpad', 'history_analysis', 'input']


In [21]:
result = conversational_agent.invoke(
    "If we start with $15,000 instead and follow the same 8% annual growth for 5 years with compound interest, how much more would we have compared to the previous scenario?"
)
print(f"Result: {result['output']}")

0: calculator({'expression': '15000 * (1 + 0.08)**5 - 14693.28'}) -> 7346.641152000009
1: final_answer({'answer': '$7346.64 more compared to the previous scenario.', 'tools_used': ['calculator']}) -> {'answer': '$7346.64 more compared to the previous scenario.', 'tools_used': ['calculator']}
Result: $7346.64 more compared to the previous scenario.


## Agent type #3: React Docstore

In [22]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.tools import tool

@tool
def Search(query: str) -> str:
    """Search Wikipedia for information about a topic."""
    try:
        wiki = WikipediaAPIWrapper()
        return wiki.run(query)
    except Exception as e:
        return f"Error searching Wikipedia: {e}"

@tool
def Lookup(term: str) -> str:
    """Look up a specific term or phrase in Wikipedia."""
    try:
        wiki = WikipediaAPIWrapper()
        return wiki.run(term)
    except Exception as e:
        return f"Error looking up term: {e}"

tools = [Search, Lookup]

In [23]:
# Create a custom agent executor for docstore tools
docstore_prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant that can search and lookup information."),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

docstore_agent_runnable = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: x.get("agent_scratchpad", [])
    }
    | docstore_prompt
    | chat_llm.bind_tools(tools)
)

docstore_name2tool = {tool.name: tool.func for tool in tools}

In [24]:
class DocstoreAgentExecutor:
    def __init__(self, max_iterations: int = 3):
        self.max_iterations = max_iterations
        self.agent = docstore_agent_runnable

    def invoke(self, input: str) -> dict:
        count = 0
        agent_scratchpad = []

        while count < self.max_iterations:
            tool_call = self.agent.invoke({
                "input": input,
                "agent_scratchpad": agent_scratchpad
            })

            agent_scratchpad.append(tool_call)

            # print(f"Agent output at iteration {count}:", tool_call)

            if not tool_call.tool_calls:
                final_answer = tool_call.content
                break

            tool_name = tool_call.tool_calls[0]["name"]
            tool_args = tool_call.tool_calls[0]["args"]
            tool_call_id = tool_call.tool_calls[0]["id"]
            tool_out = docstore_name2tool[tool_name](**tool_args)

            tool_exec = ToolMessage(
                content=f"{tool_out}",
                tool_call_id=tool_call_id
            )
            agent_scratchpad.append(tool_exec)

            print(f"{count}: {tool_name}({tool_args}) = {tool_out}...")
            count += 1

        return {"input": input, "output": final_answer}

docstore_agent = DocstoreAgentExecutor()

In [25]:
result = docstore_agent.invoke("What were Archimedes' last words?")
print(f"Result: {result['output']}")

0: Lookup({'term': 'Archimedes death'}) = Page: Archimedes' heat ray
Summary: Archimedes is purported to have invented a large scale solar furnace, sometimes described as a heat ray, and used it to burn attacking Roman ships during the Siege of Syracuse (c. 213–212 BC). It does not appear in the surviving works of Archimedes and there is no contemporary evidence for it, leading to modern scholars doubting its existence. It was an established story about Archimedes by around 500 AD, when Anthemius described a reconstruction, and it has become the subject of speculation about its plausibility.

Page: Archimedes
Summary: Archimedes of Syracuse ( AR-kih-MEE-deez; c. 287 – c. 212 BC) was an Ancient Greek mathematician, physicist, engineer, astronomer, and inventor from the city of Syracuse in Sicily. Although few details of his life are known, based on his surviving work, he is considered one of the leading scientists in classical antiquity, and one of the greatest mathematicians of all tim